# MACD strategy - BSE

In [125]:
import pandas as pd
import requests
from datetime import datetime
import plotly.graph_objects as go

In [126]:
# BSE index code for Federal Bank
# Federal back = BOM500469
# TCS = BOM532540
index_code = "BOM532540"
start_date = "01-05-2020"

In [127]:
# Convert date string to datetime object
start_date = datetime.strptime(start_date, "%d-%m-%Y")

In [128]:
# Get the data for the mentioned entity from Quandl
url = f"https://www.quandl.com/api/v3/datasets/BSE/{index_code}?api_key=JVU4wxLxNHFtSRjCvezu"

response = requests.get(url=url).json()

In [129]:
# Create a dataframe from the data for computation
df = pd.DataFrame(columns=response["dataset"]["column_names"], data=response["dataset"]["data"])

In [130]:
# Write data to csv
# df.to_csv(index_code+".csv", index=False)

In [131]:
df.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'WAP', 'No. of Shares',
       'No. of Trades', 'Total Turnover', 'Deliverable Quantity',
       '% Deli. Qty to Traded Qty', 'Spread H-L', 'Spread C-O'],
      dtype='object')

In [132]:
# Select subset of the complete data for ease of computation
df = df[["Date", "Open", "High", "Low", "Close"]]

In [133]:
# Fast length
df["MA12"] = df["Close"].ewm(span=12, adjust=False).mean()
# Slow Length
df["MA26"] = df["Close"].ewm(span=26, adjust=False).mean()

In [134]:
df["MACD"] = df["MA26"] - df["MA12"]

In [135]:
# Signal Length
df["signal"] = df["MACD"].ewm(span=9, adjust=False).mean()

In [136]:
# Step 1 for decision
df["move"] = df["MACD"]>df["signal"]
df["move"] = df["move"].apply(lambda x: float(x))

In [137]:
# Step 2 for decision
df["positions"] = df["move"].diff()

In [138]:
# Convert the date column to datetime object
df["Date"] = pd.to_datetime(df["Date"])

In [139]:
# Subset the data based on date
rev_df = df[df["Date"]>start_date]
rev_df.sort_values("Date", ascending=True, inplace=True)

<ipython-input-139-4e5c8341733b>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [140]:
def deal_stocks(df, initial_money=50000, max_buy=1, max_sell=1):
    """Function to buy or sell based on the decision column i.e. positions
    
    Arguments:
    df -> Input dataframe containing the stock data
    initial_money -> Amount of money we want to put
    max_buy -> Maximum amount of shares we want to buy
    max_sell -> Maximum amount of shares we want to sell
    """
    money = initial_money
    sell_days = []
    buy_days = []
    shares_count = 0

    def buy(day, money, shares_count):
        date = day["Date"]
        close = day["Close"]
        shares = money // day["Close"]

        if shares < 1:
            print(f"Date: {date}: Total balance: {money}, not enough money to buy {shares} shares with unit price of {close}")
        else:
            if shares > max_buy:
                units = max_buy
            else:
                units = shares

            money = money - (units*day["Close"])
            shares_count = shares_count + units
            print(f"Date {date}: Buy {units} units of shares at a price of {close}, total balance is {money}")
            buy_days.append([date, close])
        return money, shares_count

    def sell(day, money, shares_count):
        date = day["Date"]
        close = day["Close"]
        if shares_count == 0:
            print(f"Date {date}: Cannot sell anything as available shares is 0")
        else:
            if shares_count > max_sell:
                units = max_sell
            else:
                units = shares_count
            money = money + (units*day["Close"])
            shares_count = shares_count - units
            print(f"Date {date}: Sell {units} units of shares at a price of {close}, total balance is {money}")
            sell_days.append([date, close])
        return money, shares_count

    for index, row in df.iterrows():
        state = row["positions"]
        if state == 1.0:
            money, shares_count = buy(row, money, shares_count)
        elif state == -1.0:
            money, shares_count = sell(row, money, shares_count)

    total_gains = initial_money - money
    return buy_days, sell_days, total_gains

In [141]:
# Execute the deals
buy_data, sell_data, total_gain = deal_stocks(rev_df)

Date 2020-05-05 00:00:00: Cannot sell anything as available shares is 0
Date 2020-05-28 00:00:00: Buy 1 units of shares at a price of 2005.2, total balance is 47994.8
Date 2020-06-11 00:00:00: Sell 1 units of shares at a price of 2067.8, total balance is 50062.600000000006
Date 2020-07-01 00:00:00: Buy 1 units of shares at a price of 2092.55, total balance is 47970.05
Date 2020-07-16 00:00:00: Sell 1 units of shares at a price of 2233.65, total balance is 50203.700000000004
Date 2020-07-24 00:00:00: Buy 1 units of shares at a price of 2157.05, total balance is 48046.65
Date 2020-08-25 00:00:00: Sell 1 units of shares at a price of 2242.25, total balance is 50288.9
Date 2020-09-11 00:00:00: Buy 1 units of shares at a price of 2373.7, total balance is 47915.200000000004
Date 2020-09-14 00:00:00: Sell 1 units of shares at a price of 2492.3, total balance is 50407.50000000001
Date 2020-10-05 00:00:00: Buy 1 units of shares at a price of 2706.85, total balance is 47700.65000000001
Date 2020

In [142]:
print(f"If invested starting from {start_date}, then total gain would have been {total_gain}")

If invested starting from 2020-05-01 00:00:00, then total gain would have been 2680.7499999999927


In [143]:
# Creating buying and selling data dataframes
buy_df = pd.DataFrame(columns=["Date", "Close"], data=buy_data)
sell_df = pd.DataFrame(columns=["Date", "Close"], data=sell_data)

In [144]:
# Plot all the relevant values
close_chart = go.Scatter(x=rev_df["Date"], y=rev_df["Close"], mode="lines")
buy_chart = go.Scatter(x=buy_df["Date"], y=buy_df["Close"], mode="markers", marker_symbol="triangle-up", marker=dict(size=20, color="red"))
sell_chart = go.Scatter(x=sell_df["Date"], y=sell_df["Close"], mode="markers", marker_symbol="triangle-down", marker=dict(size=20, color="green"))
fig = go.Figure(data=[close_chart, buy_chart, sell_chart])
fig.update_layout(width=1200)
fig.show()